In [7]:
import os
import pandas as pd
import nltk
import en_core_web_sm
import spacy
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import line_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
import re
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')  
nlp = en_core_web_sm.load()

from IPython.display import display, HTML
import re
import string



In [8]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 5000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_column',None)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', -1)

In [9]:
audio_directory = "/Users/mmanle240/Downloads/nlp_git/Audio_Transcription/Amazon/Transcripts2"
os.chdir(audio_directory)

In [10]:
## Read in all txt files into one list

location = os.getcwd() # get present working directory location here

file_list = [] 

df_list=[]

for file in os.listdir(location):
    if file.endswith(".txt"):
        file_list.append(file[:-4])
        df = nltk.data.load(file)
        df_list.append(df)

In [11]:
# Dictionary for phrases transformation


dic={'home security system':'homesecuritysystem',\
     'security system':'securitysystem',\
     'home security':'homesecurity',
     'set top box':'settopbox',\
     'xfinity mobile':'xfinitymobile',\
     'infinity mobile':'xfinitymobile',\
     'land line':'landline',\
     'home phone':'homephone',\
     'hot spot':'hotspot',\
     'home phones':'homephone',\
     'quad play':'quadplay',\
     'double play':'doubleplay',\
     'triple play':'tripleplay',\
     'single play':'singleplay',\
     'performance pro':'performancepro',\
     'performance plus':'performanceplus',\
     'performance starter':'performancestarter',\
     'amazon prime': 'amazonprime',\
     'sling tv':'slingtv',\
     'direct tv':'directtv',\
     'high definition':'highdefinition',\
     'megabyte':'megabytes',\
     'megabits':'megabytes',\
     'premium channel':'premiumchannel',\
     'show time':'showtime'
    }
    


# Function to replace keywords in text regardless of case
def replace_all(text, dic):
    for i, j in dic.items():
        src_str  = re.compile(i, re.IGNORECASE)
        text = src_str.sub(j,text)
    return text

# Transform and Tokenize each chat 
token_chat=[]
for chat in df_list:
    chat=replace_all(chat, dic) # convert phrases into one word format according to preset dictionary 
    token_txt = line_tokenize(chat)
    token_word = [nltk.word_tokenize(txt) for txt in token_txt]
    token_chat=token_chat+(token_word)

In [12]:

# Exclude hyphen from punctuation removal list


premove = string.punctuation
premove = premove.replace("-", "") # don't remove hyphens


#Specify a list of stop words to remove
more_stops=['I','say','hello','na','gon','get','uh','right','guy','still','come','say','someth','mean','um','oh','yes','bye','great','yeah','no','sorry','ok','okay','thank','like','alright','know']
stop_words = set(nltk.corpus.stopwords.words('english')+more_stops)

# Create a vector for clean chat.
chat_clean = [None] * len(token_chat)

# Create a vector of word counts for each clean chat.
word_counts_per_chat = [None] * len(token_chat)

# Process words in all chats.
for i in range(len(token_chat)):
    # 1. Convert to lower case.
    chat_clean[i] = [chat.lower() for chat in token_chat[i]]
    
    # 2. Remove stopwords.
    chat_clean[i] = [word for word in chat_clean[i] if not word in stop_words]
    
    # 3. Remove punctuation and any non-alphabetical characters.
    chat_clean[i] = [word for word in chat_clean[i] if word not in premove]
    
    # 4. Stem words.
    #chat_clean[i] = [PorterStemmer().stem(word) for word in chat_clean[i]]
    
    # Add POS and Name entity to words
    chat_clean[i]=tree2conlltags(ne_chunk(pos_tag((chat_clean[i]))))
    
    # Record the word count per chat.
    word_counts_per_chat[i] = len(chat_clean[i])

In [13]:
# Add chat number to each row as id 
new_tpl=[]
for i in range(len(chat_clean)):
    n=i+1
    for tpl in chat_clean[i]:
        tpl_update=(n,)+tpl
        new_tpl.append(tpl_update)

In [14]:
# Set globals


# Need lower threshold that represents cheapest package. 
# Necessary to filter out noise from phone numbers, quantifiers, etc.
lower_threshold = 30

# Need upper threshold to help rule out discussion of wifi speeds. 
upper_threshold = 450

integers = ["1", "2", "3", "4", "5", "6", "7", "8", "9" ]

units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen"
    ]

tens = ["", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

scales = ["hundred", "thousand", "million", "billion", "trillion"]


doc_check_keys = ["a", "and", "dollars", "cents", "cent", "dollar"]


no_package_moce_price = {'NO PRODUCTS': 4.99}

single_package_mode_price = {'VIDEO ONLY': 49.99, 'CDV ONLY': 29.99, 'XM ONLY': 30.00, 'HSD ONLY': 39.99
                            , 'XH ONLY' : 24.99
    
}

double_package_mode_price = {'HSD/XH': 39.99 , 'HSD/CDV': 59.99 , 'CDV/XH': 19.99, 'XH/XM': 24.99,
                            'VIDEO/CDV': 74.99 , 'CDV/XM': 25.00, 'VIDEO/XM': 54.99, 'HSD/XM': 39.99 , 
                            'VIDEO/XH': 29.99 , 'VIDEO/HSD': 49.99 
}

triple_package_mode_price = {'VIDEO/CDV/XH': 24.99 , 'VIDEO/HSD/XM': 89.99 , 'HSD/CDV/XM': 59.99 ,
                            'CDV/XH/XM': 24.10 , 'VIDEO/HSD/XH': 109.99 , 'HSD/XH/XM': 39.99 , 
                            'VIDEO/HSD/CDV': 119.99 , 'HSD/CDV/XH': 59.99 , 
                            'VIDEO/CDV/XM': 54.99, 'VIDEO/XH/XM': 24.99
                            
    
}

quad_package_mode_price = {'VIDEO/HSD/XH/XM': 109.99, 'VIDEO/HSD/CDV/XH': 24.99, 'HSD/CDV/XH/XM': 29.95, 
                          'VIDEO/CDV/XH/XM': 24.99, 'VIDEO/HSD/CDV/XM': 119.99
    
}

price_dict = dict(single_package_mode_price)
price_dict.update(double_package_mode_price) 
price_dict.update(triple_package_mode_price) 
price_dict.update(quad_package_mode_price) 

master_price_dict = {19.99: 'CDV/XH',
               24.10: 'CDV/XH/XM', 
               24.99: ('XH ONLY', 'XH/XM', 'VIDEO/CDV/XH', 'VIDEO/XH/XM',
                       'VIDEO/HSD/CDV/XH', 'VIDEO/CDV/XH/XM', 'CDV/XM'), 
                29.99: ('HSD/CDV/XH/XM', 'CDV ONLY', 'VIDEO/XH', 'XM ONLY'), 
               39.99: ('HSD ONLY', 'HSD/XH', 'HSD/XM', 'HSD/XH/XM'), 
               49.99: ('VIDEO ONLY', 'VIDEO/HSD'), 
               54.99: ('VIDEO/XM', 'VIDEO/CDV/XM'), 
               59.99: ('HSD/CDV', 'HSD/CDV/XM', 'HSD/CDV/XH'), 
               74.99: 'VIDEO/CDV', 
               89.99: 'VIDEO/HSD/XM', 
               109.99: ('VIDEO/HSD/XH', 'VIDEO/HSD/XH/XM'), 
               119.99: ('VIDEO/HSD/CDV', 'VIDEO/HSD/CDV/XM')
              }
               


# price_dict_rev = {v: k for k, v in price_dict.items()}
price_list = [19.99, 24.10, 24.99, 29.99, 39.99, 49.99, 54.99, 59.99, 74.99, 89.99, 109.99, 119.99]

In [15]:

# Product related keywords
hsd=['speed','internet','megabyte','megabytes','megabits','modem','gigabytes','signal','wifi','network','router']
video=['channel','channels','set-top-box','dvr','hd','hbo','espn','box','boxes']
cdv=['landline','homephone']
xm=['mobile','lines','hotspot','spot','xfinity-mobile']
xh=['securitysystem','homesecuritysystem','homesecurity','camera','cameras']

values = ['hsd']
dictionary = dict(zip(hsd, values))

# Other Categories of keywords
stream=['streaming','stream']
ott=['netflix','hulu','roku']
hsd_tier=['blast','performance','performance-pro','performanceplus','performancestarter','gigabit']
premium_channel=['premiumchannel','hbo','espn','showtime','cinemax','movie channels']
vid_tier=['latino','dvr','hd','highdefinition']
packages=['quadplay','doubleplay','tripleplay','singleplay']
tp=['tripleplay','triple']
dp=['doubleplay','double']
qp=['quadplay','quad']


# Convert tuples to pandas dataframe
df=pd.DataFrame(new_tpl, columns=['Chat#','Word','POS','Entity']).drop(['Entity','POS'], axis=1) 

# Tag keywords based on word categoriess
df['tag'] = df['Word'].apply(lambda x: 'HSD' if x.lower() in hsd \
                             else 'VID' if x.lower() in video \
                             else 'CDV' if x.lower() in cdv \
                             else 'XH' if x.lower() in xh\
                             else 'XM' if x.lower() in xm\
                             else 'HSD_TIER' if x.lower() in hsd_tier\
                             else 'VID_TIER' if x.lower() in vid_tier\
                             else 'PACKGE' if x.lower() in packages\
                             else 'STREAM' if x.lower() in stream\
                             else 'OTT' if x.lower() in ott else 'OTHER')


df['tag2'] = df['Word'].apply(lambda x: 'HSD_TIER' if x.lower() in hsd_tier\
                             else 'VID_TIER' if x.lower() in vid_tier\
                             else 'PACKGE' if x.lower() in packages\
                             else 'STREAM' if x.lower() in stream\
                             else 'OTT' if x.lower() in ott \
                             else 'PREM_CHANNEL' if x.lower() in premium_channel \
                            else 'OTHER')

In [16]:
conversion_dict = {
'speed':'hsd','internet':'hsd','megabyte':'hsd',
'megabytes':'hsd','megabits':'hsd','modem':'hsd',
'gigabytes':'hsd','signal':'hsd','wifi':'hsd',
'network':'hsd','router': 'hsd', 
'channel':'video','channels':'video','set-top-box':'video',
'dvr':'video','hd':'video','hbo':'video','espn':'video',
'box':'video','boxes':'video',
'landline':'cdv','homephone': 'cdv', 
'mobile': 'xm','lines': 'xm','hotspot': 'xm',
'spot': 'xm','xfinity-mobile': 'xm', 
'securitysystem': 'xh','homesecuritysystem':'xh',
'homesecurity': 'xh','camera':'xh','cameras':'xh'}


In [17]:
# Determines how many sentences before and after the current one we can take in for context. 

def check_position(i, sentence_count, read_to_count):
    if i > 1:
        low = i - 2
    else: 
        low = 0
    if sentence_count - 1 - i > 1:
        high = i + 3 
    else: 
        high = sentence_count
    return([low, high])

In [18]:
# Takes account for cases where speaker is speaking improperly including. 
# 1.) Not say hundred after give hundreds place 
# 2.) Saying something like thirty nine ninety nine rather than thiry nine dollars and ninety nine cents

def missing_category_check(textnum):
    change_made = False
    cent_added = False
    prev_word = ''
    count = 0
    words = textnum.split()
    for word in words:
        if prev_word in units and word in tens and count == 1:
            word = 'hundred ' + word
            change_made = True
            words[count] = word
        elif (prev_word in units or prev_word in tens) and word in tens and count > 1:
            word = 'dollars and ' + word
            cent_added = True
            change_made = True
            words[count] = word
        prev_word = word
        count += 1

    if change_made:
        s = " ".join(words)
        if cent_added:
            s = s + " cents"
        return s
    else:
        return textnum

In [19]:
def text2int(textnum, numwords={}):
    
    # Put everything in lower case, strip commas, and call missing_category_check on textnum. 
    textnum = textnum.lower()
    textnum = textnum.replace(',', '')
    textnum = missing_category_check(textnum)

    # Made True upon finding keywords from 'descriptors'
    take_ave = False 
    waiting_for_and = False
    
    # Stores values for calculating averages with 'to' and 'between' cases.
    keep = [] 

    # List of non number keywords to look for.
    descriptors = ['a', 'to', 'between','dollars', 'cents']
      
    if not numwords:      
    # And symbolizes addition
      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):
          numwords[word] = (1, idx)
      for idx, word in enumerate(tens):
          numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):  
          numwords[word] = (10 ** (idx * 3 or 2), 0)
        
    current = result = 0
    
    # For each word test if in 'descriptors or if a number word.'
    for word in textnum.split():
        if word not in numwords and word not in descriptors:
            continue 
        # 'a' in a number clause symbolizes '1'
        if word == 'a': 
            word = 'one' 
        # 'to' compares two values. When found will calculate average of two. 
        elif word == 'to': 
            take_ave = True 
            keep.append(current) 
            current = 0
            result = 0
            continue
        # 'between' compares two values in conjunction with 'and'. When found will calculate average of two. 
        elif word == 'between':
            waiting_for_and = True
            continue
        elif waiting_for_and and word == "and":
            waiting_for_and = False
            take_ave = True
            keep.append(current)
            current = 0
            result = 0
            continue
        # Anything after the word dollars needs to be treated as cents. 
        # Missing category_check will add cents to end on proper condition if speaker hasn't.  
        elif word == 'dollars':
            result = result + current 
            current = 0
            continue
        # Makes sure cents aren't being read as dollars. 
        elif word == 'cents':
            current = current * .01
            continue

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
            
    # if 'to' or 'between' was found in clause, calculate average. Otherwise proceed normally.        
    if take_ave and keep[0] != 0: 
        return(result + current + sum(keep)) / (len(keep) + 1)
    else:
        return result + current


In [20]:
# Checks if price is within potential price range. 
def price_test(price): 
    if price < lower_threshold or price > upper_threshold: 
        return False 
    else: 
        return True

In [22]:
def price_highlight(text):  
    text = text.translate(str.maketrans('', '', string.punctuation))
    word_count = 0
    phrase = "" 
    instances = [] 
    for word in text.lower().split():
        if word in units or word in tens or word in scales or word in doc_check_keys or word in integers:
            if len(phrase) == 0: 
                phrase = word
            else: 
                phrase = phrase + " " + word  
            word_count += 1 
        else: 
            if phrase != "": 
                if word not in integers: 
                    int_price = text2int(phrase) 
                else: 
                    int_price = int(phrase)
                test_price = price_test(int_price) 
                if test_price: 
                    if phrase.split()[-1] in ('a', 'and'): 
                        phrase = phrase.rsplit(' ', 1)[0]
                    text = text.replace(phrase, "$#" + phrase + "#$")
                    print("$#" + phrase + "#$")
            phrase = "" 
            word_count = 0
    if phrase != "": 
        int_price = text2int(phrase)  
        test_price = price_test(int_price) 
        if test_price: 
            text = text.replace(phrase, "$#" + phrase + "#$") 
        
    return text

In [100]:
def extract_info(transcript_sentences):
    context = ""
    i = 0 
    output_count = 1
    transcript_context = ''
    transcript_score = 0 
    transcript_context = ''  
    read_to_count = 0 
    sentence_count = len(transcript_sentences)
    while i < sentence_count:
        context_gauge = check_position(i, sentence_count, read_to_count) 
        lower_limit = context_gauge[0] 
        upper_limit = context_gauge[1]
        #doc = nlp(transcript_sentences[i])
        for word in transcript_sentences[i].split(): 
            context = "" 
            if word in conversion_dict: 
                context = "\n\n" + context + conversion_dict[word].upper() + ":" + str(output_count) + "\n"
                word = word.upper()
                output_count += 1
                z = lower_limit 
                while z < i: 
                    context = context + transcript_sentences[z] 
                    z += 1
                    if z == i: 
                        break 
                    else: 
                        context = context + " "
                context = context + " "  
                for word in transcript_sentences[i].split():
                    if word in conversion_dict: 
                        context = context + word.upper() + " "
                    else: 
                        context = context + word + " "
                context = context + " "
                z += 1
                while z < upper_limit:
                    context = context + transcript_sentences[z] + " "
                    z += 1
                read_to_count = z
                context = price_highlight(context) 
                #context = context.strip("[").strip("]")
                transcript_context =  transcript_context + " " + context
                break 
        i += 1
    return transcript_context



In [101]:
col_names =  ['Transcript_ID', 'Chat#', 'Summary'] 
my_df  = pd.DataFrame(columns = col_names)
my_df
chat_count = 0 
for chat in df_list: 
    chat.translate(str.maketrans('', '', string.punctuation))    
    transcript_sentences = tokenizer.tokenize(chat) 
    #print(transcript_sentences)
    data = extract_info(transcript_sentences)  
    row = [file_list[chat_count], chat_count, data]
    my_df.loc[len(my_df)] = row
    chat_count += 1

$#a hundred#$
$#one hundred twenty four dollars and ninety nine cents#$
$#two hundred and fifty#$
$#one hundred and fifty#$
$#two hundred and fifty#$
$#one hundred and fifty#$
$#two hundred six dollars and seventy seven cents#$
$#one hundred and fifty#$
$#fifty dollars#$
$#fifty dollars#$
$#sixty six#$
$#a hundred and forty#$
$#a hundred and forty#$
$#1 seventy six fifty eight#$
$#a hundred and eighty five dollars#$
$#one hundred forty#$
$#two hundred twenty#$
$#one hundred fifty#$
$#one hundred forty#$
$#two hundred twenty#$
$#one hundred fifty#$
$#two hundred#$
$#two hundred and fifty#$
$#two hundred#$
$#two hundred and fifty#$
$#two hundred#$
$#two hundred and fifty#$
$#one ninety one ninety five ninety five#$
$#one eighty five ninety five#$
$#a hundred and fifty#$
$#a hundred and fifty#$
$#two fifty#$
$#fifty#$
$#ninety dollars#$
$#a hundred#$
$#a hundred#$
$#a hundred#$
$#ninety dollars#$
$#ninety four ninety nine#$
$#ninety four ninety nine#$
$#a hundred and fifty two sixty#$
$#o

In [102]:
def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))


In [103]:
pretty_print(my_df)

,Transcript_ID,Chat#,Summary
0,10041519431551386351,0,HSD1Shes looking Picks it She said ILL give you $#a hundred#$ MEGABYTES plus for thirty nine because I have twenty nine dollars a month now Not okay She said she would work in about twenty minutes HSD2They paying that I guess they painted them Its actually slower now than it was before So I need you the check and see if she actually did increase my Internet SPEED core Its same as it rolls Sure HSD3Okay Let me just check your boot file real quick Looks like you have the SPEED extreme to your two Just a moment But okay HSD4Then weLL test it again Okay And then um just so we can get a SPEED test on file Could you also run a speed test over at speed test dot extended t dot com Okay I can get to do Sure because its its back working there because the stocks are still okay HSD5Okay And then um just so we can get a speed test on file Could you also run a SPEED test over at SPEED test dot extended t dot com Okay I can get to do Sure because its its back working there because the stocks are still okay The try Ill try this again HSD6Okay Im cancelling Its a SPEED test dot d dot com Okay I guess Steve test dog at HSD7Okay Testing And its P start test Well if for some reason that um youre not achieving the INTERNET speeds that youre paying for um could be that theres some back in SIGNAL quality issues that are preventing that But uh you do have the option of just going back to what you had before Um sir I have a feeling better but you know even with every core his slow down in the last What last two three weeks
1,23505598841550185660,1,VIDEO1So what can I do for you uh customer set up for a pretty big instill today um for five security cameras And uh we got all up and going but um its a pretty wide house and I was able to get everything up and going and its still working Um the BOX was passing on pace and everything earlier but we cant get into the attic They have a huge boat in the garage uh with the pull down attic And cynicism is not the truck here to pull it out VIDEO2They have a huge boat in the garage uh with the pull down attic And cynicism is not the truck here to pull it out And uh the main next one BOXES reporting back past with warning and it wont let me complete the job And so I mean I guess I need help Complete it out VIDEO3Do you have picture on their part Yeah Yeah the BOXES go And I had pasting jobs Its just we I think theres an old amp up in the attic and we cant get really at it VIDEO4I mean it says pass It doesnt all morning Okay let let me rerun mind because the BOX and the MOCA giving them OK issues let me Okay But I havent ran in the last like maybe five or ten minutes
2,31700020911549746015,2,VIDEO1I do have the first full type package for $#one hundred twenty four dollars and ninety nine cents#$ to twenty four months I backdated it to December twenty fourth two thousand eighteen YouLL get the uh digital er CHANNEL lineup up $#two hundred and fifty#$ channels You get stars for your premium channels You get screened picks the office you over seventy thousand three movies on demand you get your blast Internet services which are $#one hundred and fifty#$ megabytes per second which I spend at home phone and limited talking Tex and the distance and you have your modem HSD2YouLL get the uh digital er channel lineup up $#two hundred and fifty#$ channels You get stars for your premium channels You get screened picks the office you over seventy thousand three movies on demand you get your blast Internet services which are $#one hundred and fifty#$ MEGABYTES per second which I spend at home phone and limited talking Tex and the distance and you have your modem U T v a doubt Theres your HD standard box HSD3Im going over this package Whatever Washington stars crap Probably once and its Its Its Its Its Its Its Its Its cable and NETWORK news Were trying to go to bed and goes way Alright Your new estimated real clean bill will be $#two hundred six dollars and seventy seven cents

In [86]:
my_df.head(25)

,Transcript_ID,Chat#,Summary
0,10041519431551386351,0,\n\nHSD1\nShes looking Picks it She said ILL give you $#a hundred#$ MEGABYTES plus for thirty nine because I have twenty nine dollars a month now Not okay She said she would work in about twenty minutes \n\nHSD2\nThey paying that I guess they painted them Its actually slower now than it was before So I need you the check and see if she actually did increase my Internet SPEED core Its same as it rolls Sure \n\nHSD3\nOkay Let me just check your boot file real quick Looks like you have the SPEED extreme to your two Just a moment But okay \n\nHSD4\nThen weLL test it again Okay And then um just so we can get a SPEED test on file Could you also run a speed test over at speed test dot extended t dot com Okay I can get to do Sure because its its back working there because the stocks are still okay \n\nHSD5\nOkay And then um just so we can get a speed test on file Could you also run a SPEED test over at SPEED test dot extended t dot com Okay I can get to do Sure because its its back working there because the stocks are still okay The try Ill try this again \n\nHSD6\nOkay Im cancelling Its a SPEED test dot d dot com Okay I guess Steve test dog at \n\nHSD7\nOkay Testing And its P start test Well if for some reason that um youre not achieving the INTERNET speeds that youre paying for um could be that theres some back in SIGNAL quality issues that are preventing that But uh you do have the option of just going back to what you had before Um sir I have a feeling better but you know even with every core his slow down in the last What last two three weeks
1,23505598841550185660,1,\n\nVIDEO1\nSo what can I do for you uh customer set up for a pretty big instill today um for five security cameras And uh we got all up and going but um its a pretty wide house and I was able to get everything up and going and its still working Um the BOX was passing on pace and everything earlier but we cant get into the attic They have a huge boat in the garage uh with the pull down attic And cynicism is not the truck here to pull it out \n\nVIDEO2\nThey have a huge boat in the garage uh with the pull down attic And cynicism is not the truck here to pull it out And uh the main next one BOXES reporting back past with warning and it wont let me complete the job And so I mean I guess I need help Complete it out \n\nVIDEO3\nDo you have picture on their part Yeah Yeah the BOXES go And I had pasting jobs Its just we I think theres an old amp up in the attic and we cant get really at it \n\nVIDEO4\nI mean it says pass It doesnt all morning Okay let let me rerun mind because the BOX and the MOCA giving them OK issues let me Okay But I havent ran in the last like maybe five or ten minutes
2,31700020911549746015,2,\n\nVIDEO1\nI do have the first full type package for $#one hundred twenty four dollars and ninety nine cents#$ to twenty four months I backdated it to December twenty fourth two thousand eighteen YouLL get the uh digital er CHANNEL lineup up $#two hundred and fifty#$ channels You get stars for your premium channels You get screened picks the office you over seventy thousand three movies on demand you get your blast Internet services which are $#one hundred and fifty#$ megabytes per second which I spend at home phone and limited talking Tex and the distance and you have your modem \n\nHSD2\nYouLL get the uh digital er channel lineup up $#two hundred and fifty#$ channels You get stars for your premium channels You get screened picks the office you over seventy thousand three movies on demand you get your blast Internet services which are $#one hundred and fifty#$ MEGABYTES per second which I spend at home phone and limited talking Tex and the distance and you have your modem U T v a doubt Theres your HD standard box \n\nHSD3\nIm going over this package Whatever Washington stars crap Probably once and its Its Its Its Its Its Its Its Its cable and NETWORK news Were trying to go to bed and goes way Alright Your new

In [104]:
save_directory = "/Users/mmanle240/Downloads/nlp_shared_git/"
os.chdir(save_directory)

In [ ]:
#my_df.to_csv('new_view.csv')

In [105]:
from pandas import ExcelWriter

writer = ExcelWriter('PythonExport3.xlsx')
workbook  = writer.book
wrap_format = workbook.add_format({'text_wrap': True})
my_df.to_excel(writer,'Sheet1')


writer.save()